# Configuracion y Funciones

### Configuracion

In [ ]:
excel_file = "" ### Rellenar
sheet_name = "" ### Rellenar
search_column = "correo"
split_n = 100
start, end = (0, 50)  
typing_delay = (0.02, 0.05)  # Tiempo minimo y máximo entre cada letra para simular la escritura humana
deep_extraction = True # Si es True, extrae todos los datos, si es False, tambien las cajas de abajo
time_sleep = 1  # Minimo 1, Tiempo de espera entre cada acción (Aumentar si es necesario por rendimiento de maquina o internet)

auto_login = True  # Si es True, inicia sesión automáticamente
if auto_login:
	correo = "" ### Rellenar
	user = "" ### Rellenar
	password = "" ### Rellenar
 
display_correo, display_results, zoom  = True, True, True

prueba_dataset = False
if prueba_dataset:
	excel_file, sheet_name = "pruebas.xlsx", "Sheet1"
	split_n = 5
	start, end = (0, 3)

pruebas = False
if pruebas:
	excel_file, sheet_name = "pruebas.xlsx", "Sheet1"
	__name__ = "__pruebas__"
else:
	__name__ = "__main__"


### Nombre del archivo

## Funciones

### Funcion de apertura del excel

In [ ]:
import pandas as pd
def open_excel_file(file_path , sheet_name) -> pd.DataFrame:
    """Abre un archivo de Excel y devuelve el objeto de la hoja especificada."""
    try:
        # Lee el archivo de Excel y devuelve el DataFrame de la hoja especificada
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        return df
    except:
        # Si no se especifica una hoja, selecciona la hoja activa
        print("\033[33mError: No se pudo abrir la hoja especificada. Abriendo la primera hoja.\033[0m")
        df = pd.read_excel(file_path, sheet_name=0, engine='openpyxl')
        return df
    
if __name__ == "__pruebas__":
    display(open_excel_file(excel_file, sheet_name).head(5))


### Funcion de division de excels

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import concurrent.futures
import re
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

def split_excel(df: pd.DataFrame, path: str, n: int) -> None:
    """
    Divide un DataFrame en 'n' partes y guarda cada parte en un archivo Excel, utilizando:
      - np.array_split sobre un array de NumPy para evitar warnings por métodos deprecados.
      - pathlib para la gestión de rutas.
      - Escritura paralela para optimizar la operación I/O.
    
    Args:
        df (pd.DataFrame): DataFrame original.
        path (str): Ruta donde se guardarán los archivos divididos.
        n (int): Número de partes en que se dividirá el DataFrame.
    
    Raises:
        TypeError: Si alguno de los parámetros no es del tipo esperado.
        ValueError: Si n es menor o igual a 0.
    """
    # Verificación de tipos
    if not isinstance(df, pd.DataFrame):
        raise TypeError("El parámetro 'df' debe ser un DataFrame de pandas.")
    if not isinstance(path, str):
        raise TypeError("El parámetro 'path' debe ser una cadena de texto.")
    if not isinstance(n, int):
        raise TypeError("El parámetro 'n' debe ser un entero.")
    if n <= 0:
        raise ValueError("El número de partes debe ser mayor a 0.")
    
    # Crear la carpeta destino usando pathlib
    output_path = Path(path)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Limpiar archivos Excel existentes en el directorio
    for excel_file in output_path.glob("*.xlsx"):
        try:
            excel_file.unlink()  # Eliminar archivo
        except Exception as e:
            print(f"No se pudo eliminar el archivo {excel_file}: {e}")
    
    # Convertir el DataFrame a un array para evitar warnings relacionados a métodos obsoletos
    arr = df.to_numpy()
    # Dividir el array en 'n' partes
    arr_parts = np.array_split(arr, n)
    
    # Función auxiliar para escribir un DataFrame en Excel
    def write_excel(file_path: Path, part_df: pd.DataFrame):
        part_df.to_excel(file_path, index=False)
    
    # Usar un ThreadPoolExecutor para escribir los archivos en paralelo
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for idx, part in enumerate(arr_parts, start=1):
            # Si la parte está vacía, se omite
            if part.size == 0:
                continue
            # Reconstruir el DataFrame, preservando nombres de columnas
            part_df = pd.DataFrame(part, columns=df.columns)
            file_name = output_path / f"df_part_{idx}.xlsx"
            futures.append(executor.submit(write_excel, file_name, part_df))
        # Esperar a que finalicen todas las escrituras
        concurrent.futures.wait(futures)

# Ejemplo de uso:
if __name__ == '__pruebas__':
    # Crear un DataFrame de ejemplo
    data = {'col1': range(1000), 'col2': range(1000, 2000)}
    df_example = pd.DataFrame(data)
    
    # Llamar a la función para dividir y guardar el DataFrame en x partes
    split_excel(df_example, "output/parts", split_n)
	

### Funcion creacion de carpetas

In [ ]:
import os
import shutil

def create_folders(*paths):
    """
    Creates one or more folders at the specified paths. If any folder already exists,
    it will be deleted first and then recreated.
    
    Args:
        *paths: One or more path strings to create/recreate folders.
               Can be individual arguments or a list/tuple of paths.
    """
    # Convert paths to flat list if any argument is a list or tuple
    all_paths = []
    for path in paths:
        if isinstance(path, (list, tuple)):
            all_paths.extend(path)
        else:
            all_paths.append(path)
    
    for path in all_paths:
        # Delete the folder if it already exists
        if os.path.exists(path):
            shutil.rmtree(path)
        
        # Create the folder
        os.makedirs(path)

if __name__ == "__pruebas__":
    create_folders("output/parts", "output/filled")


### Funcion de apertura del driver

In [ ]:
import time
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options


def get_stable_driver() -> uc.Chrome:
	"""
	Inicializa undetected-chromedriver con opciones optimizadas.
	
	Esta función abre el navegador sin iniciar sesión en ningún sitio. Se acepta un usuario opcional
	para posibles usos posteriores, pero no se utiliza para iniciar sesión.
	
	Returns:
		uc.Chrome: Instancia del driver.
	"""
	options = Options()
	options.add_argument("--disable-blink-features=AutomationControlled")
	options.add_argument("--no-sandbox")
	options.add_argument("--disable-dev-shm-usage")
	
	# Inicializar undetected-chromedriver
	driver = uc.Chrome(options=options)
	driver.implicitly_wait(10)
	
	return driver

# Ejemplo de uso:
if __name__ == '__pruebas__':
	# Iniciar undetected-chromedriver sin iniciar sesión
	driver = get_stable_driver()
	
	# Espera para ver el navegador abierto
	time.sleep(3)
	

### Funcion de hacer pantalla grande y zoom out

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

def make_full_screen(driver: uc.Chrome) -> None:
	"""
	Coloca la ventana del navegador en pantalla completa y maximiza la ventana.
	
	Args:
		driver (uc.Chrome): Instancia del driver.
	"""
	driver.fullscreen_window()
	driver.maximize_window()
	time.sleep(2)

def zoom_out(driver: uc.Chrome, percentage: int = 75) -> None:
	"""
	Ajusta el zoom del navegador a un porcentaje específico.
	
	Args:
		driver (uc.Chrome): Instancia del driver.
		percentage (int): Porcentaje de zoom deseado. Default es 75.
	"""
	body = driver.find_element(By.TAG_NAME, 'body')
	for _ in range(100 - percentage):
		body.send_keys(Keys.CONTROL, Keys.SUBTRACT)
	time.sleep(2)


### Funcion escribir en elemento

In [ ]:
import time
import random
from selenium.webdriver.remote.webelement import WebElement

def write_keys(text: str, element: WebElement, delay_range: tuple[float, float] = (0.02, 0.05)) -> None:
	"""
	Envía una cadena de caracteres a un elemento de Selenium simulando escritura humana.
	
	Esta función recorre cada carácter de la cadena y lo envía al elemento especificado,
	añadiendo un retraso aleatorio configurable entre cada tecla para imitar la escritura natural.
	
	Args:
		text (str): La cadena de caracteres a enviar.
		element (WebElement): El elemento de Selenium (por ejemplo, un campo de búsqueda) en el que se enviarán las teclas.
		delay_range (tuple[float, float], opcional): Rango de tiempos (mínimo, máximo) en segundos para el retraso entre cada tecla.
													  El valor por defecto es (0.02, 0.05).
	"""
	min_delay, max_delay = delay_range
	for char in text:
		element.send_keys(char)
		time.sleep(random.uniform(min_delay, max_delay))

### Funcion de inicio de sesion

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def iniciar_sesion(driver, correo, user, password) -> None:
	"""
	Inicia sesión en la página de contactos de Google utilizando las credenciales proporcionadas.
	
	Esta función interactúa con los elementos de la página de inicio de sesión para enviar el correo,
	el usuario y la contraseña, simulando la escritura humana para cada campo.
	
	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.
		correo (str): Dirección de correo electrónico del usuario.
		user (str): Nombre de usuario del sistema.
		password (str): Contraseña del usuario.
	"""
	# Mandar las credenciales
	correo_input_element = driver.find_element(By.XPATH, '//*[@id="identifierId"]')
	# Escribir el correo simulando la escritura del correo carácter por carácter
	write_keys(correo, correo_input_element, delay_range=typing_delay)

	# Darle a siguiente
	correo_input_element.send_keys(Keys.ENTER)

	# Esperamos a que redirija a la página de inicio de sesión
	user_input_element = WebDriverWait(driver, 10).until(
		EC.presence_of_element_located((By.XPATH, '//*[@id="username"]'))
	)
	write_keys(user, user_input_element, delay_range=typing_delay)

	# Ponemos la contraseña
	password_input_element = driver.find_element(By.XPATH, '//*[@id="password"]')
	write_keys(password, password_input_element, delay_range=typing_delay)
	time.sleep(1)
	password_input_element.send_keys(Keys.ENTER)

	two_factor_auth = str(input("Presiona Enter después de rellenar el 2FA..."))
	try:
		if two_factor_auth not in ["", " ", None]:
			two_factor_auth_element = WebDriverWait(driver, 10).until(
				EC.presence_of_element_located((By.ID, "otp_mail"))
			)
			write_keys(two_factor_auth, two_factor_auth_element)
			two_factor_auth_element.send_keys(Keys.ENTER)
	except: pass

	try:
		# Clicamos en continuar
		continuar = WebDriverWait(driver, 10).until(
			EC.element_to_be_clickable((By.XPATH, '//*[@id="yDmH0d"]/div[1]/div[1]/div[2]/div/div/div[3]/div/div[1]/div/div/button'))
		)
		continuar.click()
	except: pass


if __name__ == '__pruebas__':
	driver = get_stable_driver()
	make_full_screen(driver)
	zoom_out(driver, percentage=75)
	driver.get("https://contacts.google.com/")
	iniciar_sesion(driver, correo, user, password)


### Funcion de busqueda de contacto

In [ ]:
def contact_search(driver: uc.Chrome, search_value: str) -> None:
	"""
	Busca un contacto en Google Contacts.
	
	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.
		search_value (str): Valor a buscar.
	"""
	search_bar_element = WebDriverWait(driver, 10).until(
		EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[2]/div[2]/form/div/div/div/div/div/div[1]/input[2]'))
	)
	search_bar_element.click()
	search_bar_element.clear()  # Aseguramos que esté vacía antes de escribir
	person = str(person)
	write_keys(search_value, search_bar_element, delay_range=typing_delay)
	time.sleep(time_sleep)

	search_bar_element.send_keys(Keys.ENTER)
	time.sleep(time_sleep)
	

### Funcion de limpieza de emails

In [ ]:
def get_clean_emails_from_df(df: pd.DataFrame, column_name: str, display_results = True) -> list[str]:
	"""
	Obtiene una lista de correos electrónicos de un DataFrame, manejando valores nulos,
	eliminando espacios en blanco y seleccionando preferentemente correos que contengan "BBVA".
	
	Args:
		df (pd.DataFrame): DataFrame que contiene los correos electrónicos.
		column_name (str): Nombre de la columna que contiene los correos electrónicos.
	
	Returns:
		list[str]: Lista de correos electrónicos procesados.
	"""
	emails = []
	
	for value in df[column_name]:
		if pd.isna(value) or value == "" or value == " " or value == "nan" or value == "N/A" or value == "NaN" or pd.isnull(value):
			emails.append("")
		else:
			# Dividir por saltos de línea por si hay múltiples correos
			email_parts = str(value).split("\n")
			
			# Eliminar espacios en blanco de cada parte
			email_parts = [part.strip() for part in email_parts]
			
			# Buscar un correo que contenga "BBVA"
			bbva_email = None
			for part in email_parts:
				if "BBVA" in part.upper():
					bbva_email = part
					break
			
			# Si encontramos un correo con BBVA, usamos ese, sino el primero
			emails.append(bbva_email if bbva_email else email_parts[0])
	if display_results:
		print(emails)
	
	return emails

if __name__ == '__pruebas__':
	import pandas as pd
	df = pd.read_excel(excel_file, sheet_name=sheet_name)
	
	# Ver las diferencias entre los correos originales y los procesados
	print("\033[1mCorreos originales vs procesados\033[0m")
	print("--------------------------------")
	print("\033[1mCorreos originales\033[0m")
	print(len(df))
	print(list(df[search_column]))
	print("--------------------------------")
	print("\033[1mCorreos limpiados\033[0m")
	print(len(get_clean_emails_from_df(df, search_column, display_results=False)))
	print(get_clean_emails_from_df(df, search_column, display_results=False))
	print("--------------------------------")



### Funcion de busqueda de contacto

In [ ]:
def search_contact(driver: uc.Chrome, text: str) -> bool:
	"""
	Realiza la búsqueda de un contacto en Google Contacts.
	
	Esta función utiliza el driver de Selenium para ubicar la barra de búsqueda, 
	limpia su contenido, escribe el texto proporcionado simulando la escritura humana 
	y activa la búsqueda. Devuelve True si la búsqueda se ejecuta correctamente; 
	en caso de que el texto sea None o vacío, devuelve None.
	
	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.
		text (str): Texto a buscar. Debe ser un string no vacío.
	
	Returns:
		bool or None: True si se ejecuta la búsqueda correctamente, None si el texto es None o vacío.
	"""
	if text is None or text == "":
		return None

	try:
		search_bar_element = WebDriverWait(driver, 10).until(
		EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[2]/div[2]/form/div/div/div/div/div/div[1]/input[2]'))
		)
		search_bar_element.click()
		search_bar_element.clear()  # Aseguramos que esté vacía antes de escribir
		write_keys(text, search_bar_element, delay_range=typing_delay)
		time.sleep(time_sleep)
		# Esperamos a que aparezcan las sugerencias
		try:
			# Mandamos la tecla de abajo para seleccionar la primera sugerencia
			search_bar_element.send_keys(Keys.ARROW_DOWN)
			time.sleep(0.1)
			search_bar_element.send_keys(Keys.ENTER)
		except:
			return False
	except:
		return False

	return True

if __name__ == '__pruebas__':
	for i in range(58,65):
		driver.get("https://contacts.google.com/")
		time.sleep(time_sleep)
		print("\033[32mBuscando contacto:", df["correo_cleaned"].iloc[i], "\033[0m")
		print(search_contact(driver, df["correo_cleaned"].iloc[i]))
		time.sleep(time_sleep)

### Funciones de nulo y error

In [ ]:
def is_null_page(driver: uc.Chrome) -> bool:
	"""
	Verifica si la página actual es la de "No se encontraron resultados".

	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.

	Returns:
		bool: True si se encontró el mensaje "No tienes ningún contacto que coincida", False en caso contrario.
	"""
	# Ajustamos temporalmente el implicit wait a 'time_sleep' para acelerar la búsqueda
	driver.implicitly_wait(time_sleep)
	try:
		elementos = driver.find_elements(By.CSS_SELECTOR, 'div[jsname="bN97Pc"] div.YalTkb')
		return True if  elementos else False
	finally:
		# Restauramos el implicit wait al valor por defecto (por ejemplo, 10 segundos)
		driver.implicitly_wait(10)

def is_error_page(driver: uc.Chrome) -> bool:
	"""
	Verifica si la página actual es la de error (por ejemplo, un error 400).

	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.

	Returns:
		bool: True si se encontró el contenedor de error y su contenido coincide, False en caso contrario.
	"""
	# Ajusta temporalmente el implicit wait a time_sleep para que la búsqueda sea rápida
	driver.implicitly_wait(time_sleep)
	try:
		error_elements = driver.find_elements(By.XPATH, '//*[@id="af-error-container"]')
		if error_elements:
			return True if error_elements else False

		return False
	finally:
		# Restaura el implicit wait al valor por defecto (por ejemplo, 10 segundos)
		driver.implicitly_wait(10)

if __name__ == '__pruebas__':
	# Da Error
	driver.get("https://contacts.google.com/")
	time.sleep(time_sleep)
	search_contact(driver, """adrian.hernandez@nfq.esADRIAN.HERNANDEZ3.CONTRACTOR@BBVA.COM""")
	print("Es nulo:", is_null_page(driver))
	print("Da error:", is_error_page(driver), "\n")    

	# Es correcto
	driver.get("https://contacts.google.com/")
	time.sleep(time_sleep)
	search_contact(driver, df["correo_cleaned"].iloc[58])
	print("Es nulo:", is_null_page(driver))
	print("Da error:", is_error_page(driver), "\n")   
	
	# Es Nulo
	driver.get("https://contacts.google.com/")
	time.sleep(time_sleep)
	search_contact(driver, df["correo_cleaned"].iloc[59])
	print("Es nulo:", is_null_page(driver))
	print("Da error:", is_error_page(driver))   



### Funcion de extraer empresa

In [ ]:
from bs4 import BeautifulSoup

def extract_contact_info(driver: uc.Chrome) -> dict:
	"""
	Extrae información detallada de un contacto desde Google Contacts.
	
	Esta función utiliza Selenium y BeautifulSoup para analizar la estructura HTML 
	de la página de detalles de un contacto en Google Contacts y extraer información
	relevante como el nombre, género, tipo de contacto, cargo y empresa.
	
	Args:
		driver (uc.Chrome): Instancia del driver de Selenium con una página de 
						   detalles de contacto de Google Contacts ya cargada.
						   
	Returns:
		dict: Diccionario con la siguiente información del contacto:
			- "Nombre": Nombre del contacto
			- "Genero": Género del contacto
			- "Tipo de contacto": Clasificación del contacto
			- "Cargo": Posición o cargo que ocupa
			- "Empresa": Empresa a la que pertenece
			
	Note:
		Si algún campo no se encuentra, su valor será "NA".
		La función espera 2 segundos para encontrar el contenedor principal.
	"""
	# Valores por defecto
	info = {
		"Nombre": "NA",
		"Genero": "NA",
		"Tipo de contacto": "NA",
		"Cargo": "NA",
		"Empresa": "NA"
	}
	
	try:
		# Espera a que el contenedor principal esté presente
		contact_element = WebDriverWait(driver, 2).until(
			EC.presence_of_element_located((By.CSS_SELECTOR, "div.NFixgb"))
		)
	except Exception as e:
		print("No se encontró el contenedor de información de contacto:", e)
		return info

	# Parsear el HTML del contenedor con BeautifulSoup
	soup = BeautifulSoup(contact_element.get_attribute("innerHTML"), "html.parser")
	
	# Extraer nombre
	nombre_tag = soup.select_one("div.qhML4e")
	if nombre_tag:
		info["Nombre"] = nombre_tag.get_text(strip=True)
	
	# Extraer género
	genero_tag = soup.select_one("div.AyiB div.uVSYQe")
	if genero_tag:
		info["Genero"] = genero_tag.get_text(strip=True)
	
	# Extraer información de la empresa (tipo, cargo, empresa)
	company_div = soup.find("div", class_="b81mX")
	if company_div:
		# Separa el texto utilizando el separador "•"
		# El parámetro separator de get_text() ayuda a incluir el separador entre elementos
		partes = [parte.strip() for parte in company_div.get_text(separator="•", strip=True).split("•") if parte.strip()]
		# Si se tienen al menos tres partes, se asignan en orden
		if len(partes) >= 3:
			info["Tipo de contacto"], info["Cargo"], info["Empresa"] = partes[:3]
		else:
			if len(partes) > 0:
				info["Tipo de contacto"] = partes[0]
			if len(partes) > 1:
				info["Cargo"] = partes[1]
			if len(partes) > 2:
				info["Empresa"] = partes[2]
	
	return info


if __name__ == '__pruebas__':
	driver.get("https://contacts.google.com/person/110725874616136377172")
	time.sleep(time_sleep)
	display(extract_contact_info(driver))
	
	

### Funciones de scroll_down y de clickar los botones 'más'

In [ ]:
def click_more_button(driver: uc.Chrome) -> None:
	"""
	Encuentra y hace clic en todos los botones que tengan el texto 'Más' o 'More'.
	"""
	# Utilizamos un único XPath para buscar ambos textos
	xpath = "//span[text()='Más' or text()='More']"
	
	# Buscamos botones que sean visibles y estén habilitados
	more_buttons = driver.find_elements(By.XPATH, xpath)
	more_buttons = [btn for btn in more_buttons if btn.is_enabled() and btn.is_displayed()]
	
	# Si se encuentra 0 o 1 botón, esperamos y volvemos a buscar sin filtrar por visibilidad
	if len(more_buttons) <= 1:
		time.sleep(0.5)
		more_buttons = driver.find_elements(By.XPATH, xpath)
		more_buttons = [btn for btn in more_buttons if btn.is_enabled()]
	
	# Iteramos y clicamos cada botón, ignorando errores individuales
	for btn in more_buttons:
		try:
			btn.click()
		except Exception:
			pass
		time.sleep(0.2)
		

def scroll_down(driver: uc.Chrome) -> None:
	"""
	Desplaza la vista hacia abajo de la página actual utilizando un elemento scrolleable.

	La función localiza elementos con atributo "id" y utiliza uno de ellos para desplazar la vista
	mediante la ejecución de código JavaScript. En caso de que el primer intento falle, se intenta
	desplazar la vista utilizando otro elemento alternativo.

	Args:
		driver (uc.Chrome): Instancia del driver de Selenium.
	"""
	try:
		# Buscamos los elementos que sean scrolleables
		scrollable_elements = driver.find_elements(By.XPATH, "//*[@id]")
		# print(f"Se han encontrado {len(scrollable_elements)} elementos con id")
		# print(scrollable_elements[39].get_attribute("id"))
		driver.execute_script("arguments[0].scrollIntoView();", scrollable_elements[39])
	except:
		scrollable_elements = driver.find_elements(By.XPATH, "//*[@id]")
		driver.execute_script("arguments[0].scrollIntoView();", scrollable_elements[27])

if __name__ == '__pruebas__':
    driver.get("https://contacts.google.com/person/110725874616136377172")
    time.sleep(time_sleep)
    click_more_button(driver)
    time.sleep(time_sleep)
    scroll_down(driver)



### Funciones antiguas (deprecadas)

In [ ]:
# import re
# from typing import Dict
# def extract_dir_info(driver):

# 	def extraer_informacion_detallada(elemento: WebElement) -> Dict[str, str]:
# 		"""
# 		Extrae información de manera precisa usando la estructura jerárquica del HTML.
# 		"""
		
# 		informacion = {}
		
# 		try:
# 			# Buscar solo en el contenedor principal de datos
# 			contenedor_principal = elemento.find_element(By.CSS_SELECTOR, '.jHzrGd')
# 			secciones = contenedor_principal.find_elements(By.CSS_SELECTOR, '.CCWFDf')
# 		except:
# 			return informacion  # Si no hay contenedor, retornar vacío
		
# 		for seccion in secciones:
# 			try:
# 				icono = seccion.find_element(By.CSS_SELECTOR, '.google-material-icons').text.strip()
# 				contenedor_dato = seccion.find_element(By.CSS_SELECTOR, '.RjPeQd')
# 			except:
# 				continue
			
# 			if icono == 'mail':
# 				try:
# 					informacion['email'] = contenedor_dato.find_element(By.CSS_SELECTOR, 'div[aria-label^="Correo electrónico"]').text.strip()
# 				except: pass
				
# 			elif icono == 'maps':
# 				try:
# 					direccion = contenedor_dato.find_element(By.CSS_SELECTOR, 'a[aria-label^="Dirección"]').text
# 					informacion['direccion'] = direccion.replace('\n', ', ')
# 				except: pass
				
# 			elif icono == 'location_on':
# 				try:
# 					texto = contenedor_dato.find_element(By.CSS_SELECTOR, '.urwqv').text
# 					informacion['id_oficina'] = texto.split()[0]
# 				except: pass
				
# 			elif icono == 'view_list':
# 				try:
# 					texto = contenedor_dato.find_element(By.CSS_SELECTOR, '.urwqv').text
# 					informacion['id_externo'] = texto.split('•')[0].strip()
# 				except: pass
		
# 		return informacion
	
# 	elementos = driver.find_elements(By.CSS_SELECTOR, ".aHij0b-WsjYwc.j3urvf")
# 	resultados = [extraer_informacion_detallada(e) for e in elementos]
# 	# Filtrar diccionarios vacíos
# 	return [r for r in resultados if r]

# def extract_user_data(driver):

# 	def camel_a_snake(nombre: str) -> str:
# 		"""Convierte camelCase a snake_case"""
# 		nombre = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', nombre)
# 		return re.sub('([a-z0-9])([A-Z])', r'\1_\2', nombre).lower()

# 	def extraer_informacion_detallada(elemento: WebElement) -> dict:
# 		"""
# 		Extrae datos estructurados de la sección ldapUserData
# 		"""
# 		datos = {}
		
# 		try:
# 			contenedor_principal = elemento.find_element(By.CSS_SELECTOR, '.jHzrGd')
# 			grupos = contenedor_principal.find_elements(By.CSS_SELECTOR, '.EwXTZb')
# 		except:
# 			return datos
		
# 		for grupo in grupos:
# 			filas = grupo.find_elements(By.CSS_SELECTOR, '.jnkpVe')
# 			for fila in filas:
# 				try:
# 					clave_elemento = fila.find_element(By.CSS_SELECTOR, '.LEXslc')
# 					valor_elemento = fila.find_element(By.CSS_SELECTOR, '.BK9cTe')
					
# 					clave = clave_elemento.text.strip().rstrip(':')
# 					valor = valor_elemento.text.strip()
					
# 					if clave:
# 						clave_normalizada = camel_a_snake(clave)
# 						datos[clave_normalizada] = valor
# 				except:
# 					continue
		
# 		return datos

# 	elementos = driver.find_elements(By.CSS_SELECTOR, ".aHij0b-WsjYwc.j3urvf")
# 	resultados = [extraer_informacion_detallada(e) for e in elementos]
# 	return [r for r in resultados if r]

### Funcion de extracion de contactos deep

In [ ]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def extract_dir_info(driver):
    """
    Extract contact directory information from Google Contacts page.
    
    This function scrapes the contact's primary information such as email,
    physical address, organization details, and external ID from the 
    Google Contacts interface.
    
    Args:
        driver: Selenium WebDriver instance with loaded contact page
        
    Returns:
        dict: Dictionary containing extracted contact information with keys like
              'email', 'dirección', 'organización', and 'id_externo'
              Returns None if the container element cannot be found
              
    Raises:
        NoSuchElementException: Handled internally when elements are not found
    """
    try:
        contenedor = driver.find_element(By.CSS_SELECTOR, 'div.aHij0b-WsjYwc.j3urvf')
        datos = {}
        
        # Extraer secciones CCWFDf
        secciones = contenedor.find_elements(By.CLASS_NAME, 'CCWFDf')
        
        for seccion in secciones:
            try:
                # Determinar tipo de dato por el icono
                icono = seccion.find_element(By.CSS_SELECTOR, 'i.google-material-icons').get_attribute('innerHTML')
                
                if icono == 'mail':
                    # Extraer email
                    email = seccion.find_element(By.CLASS_NAME, 'W7Nbnf').text
                    datos['email'] = email.strip()
                    
                elif icono == 'maps':
                    # Extraer dirección
                    direccion = seccion.find_element(By.CSS_SELECTOR, 'a.W7Nbnf').text.replace('\n', ' ')
                    datos['dirección'] = direccion.strip()
                    
                elif icono == 'location_on':
                    # Extraer organización
                    org_text = seccion.find_element(By.CLASS_NAME, 'urwqv').text
                    org_parts = org_text.split('•')
                    datos['organización'] = org_parts[0].strip()
                    
                elif icono == 'view_list':
                    # Extraer ID externo
                    id_text = seccion.find_element(By.CLASS_NAME, 'urwqv').text
                    id_parts = id_text.split('•')
                    datos['id_externo'] = id_parts[0].strip()
                    
            except NoSuchElementException:
                continue

        return datos
    
    except NoSuchElementException:
        return None

def extract_user_data(driver):
    """
    Extract detailed user profile data from Google Contacts interface.
    
    This function scrapes all key-value pairs from the user profile section, 
    including personal details, professional information, and custom fields 
    defined in the contact record.
    
    Args:
        driver: Selenium WebDriver instance with loaded contact page
        
    Returns:
        dict: Dictionary containing all key-value pairs from the contact's profile,
              with keys representing field names and values representing their content.
              Returns None if the container element cannot be found
              
    Raises:
        NoSuchElementException: Handled internally when elements are not found
    """
    try:
        # Localizar el contenedor principal
        contenedor = driver.find_element(By.CSS_SELECTOR, 'div[jsname="lYIbdc"]')
        
        # Diccionario para almacenar los datos
        datos = {}
        
        # Extraer todos los elementos jnkpVe que contienen los pares clave-valor
        elementos = contenedor.find_elements(By.CLASS_NAME, 'jnkpVe')
        
        for elemento in elementos:
            try:
                # Extraer clave y valor
                clave = elemento.find_element(By.CLASS_NAME, 'LEXslc').text
                valor_elemento = elemento.find_element(By.CLASS_NAME, 'BK9cTe')
                valor = valor_elemento.text if valor_elemento.text.strip() != '' else None
                
                # Agregar al diccionario solo si hay valor
                if valor is not None:
                    datos[clave] = valor
            except NoSuchElementException:
                continue
        
        return datos
    
    except NoSuchElementException:
        return None


def extract_search_fields(driver):
    """
    Extract additional search fields and metadata from Google Contacts.
    
    This function targets the specific search fields section of a contact record,
    which may contain supplementary information not included in the main profile
    or directory sections, such as organizational hierarchy, regional settings,
    or system identifiers.
    
    Args:
        driver: Selenium WebDriver instance with loaded contact page
        
    Returns:
        dict: Dictionary containing additional search field data as key-value pairs.
              Returns None if the container element cannot be found
              
    Raises:
        NoSuchElementException: Handled internally when elements are not found
    """
    try:
        # Localizar el contenedor principal por su clase y jsname
        contenedor = driver.find_element(By.CSS_SELECTOR, 'div.j3urvf.jVwmLb[jsname="lYIbdc"]')
        
        datos = {}
        
        # Extraer todos los elementos jnkpVe dentro de EwXTZb
        elementos = contenedor.find_elements(By.CSS_SELECTOR, '.EwXTZb .jnkpVe')
        
        for elemento in elementos:
            try:
                clave = elemento.find_element(By.CLASS_NAME, 'LEXslc').text
                valor = elemento.find_element(By.CLASS_NAME, 'BK9cTe').text
                if valor:  # Solo agregar si hay valor
                    datos[clave] = valor.strip()
            except NoSuchElementException:
                continue
        
        return datos
    
    except NoSuchElementException:
        return None

if __name__ == '__pruebas__':
    driver.get("https://contacts.google.com/person/110725874616136377172")
    time.sleep(time_sleep)
    click_more_button(driver)
    time.sleep(time_sleep)
    scroll_down(driver)
    display("Contact_info:",extract_contact_info(driver))
    display("Directory_info:",extract_dir_info(driver))
    display("User_data_info:",extract_user_data(driver))
    display("Search_field_info:",extract_search_fields(driver))




### Extraccion de toda la info de un contacto

In [ ]:
def scrape_contact(driver, usuario: str) -> dict:
    """
    Realiza la búsqueda de un contacto en Google Contacts y extrae información relevante.
    """
    driver.get("https://contacts.google.com/")
    resultados = {}
    search_contact(driver, usuario)
    time.sleep(time_sleep)    
    if is_null_page(driver):
        print("\033[31mNo se encontraron resultados o hubo un error.\033[0m")
        return {"is_null": True}
    if is_error_page(driver):
        print("\033[31mHubo un error al buscar el contacto.\033[0m")
        return {"is_error": True}
    if deep_extraction:
        click_more_button(driver)
        time.sleep(time_sleep)
        scroll_down(driver)
        time.sleep(time_sleep)
    # Extraer información del usuario haciendo update de resultados
    resultados.update(extract_contact_info(driver))
    if deep_extraction:
        resultados.update(extract_dir_info(driver))
        resultados.update(extract_user_data(driver))
    resultados.update(extract_dir_info(driver))
    resultados.update(extract_user_data(driver))
    resultados.update(extract_search_fields(driver))
    return resultados


if __name__ == '__pruebas__':
    df_pruebas = pd.read_excel(excel_file, sheet_name=sheet_name)
    df_pruebas["correo_cleaned"] = get_clean_emails_from_df(df_pruebas, search_column, display_results=False)
    results = []
    for n in range(1, 5):
        if df_pruebas['correo_cleaned'].iloc[n] in ["", " ", None, "nan", "N/A", "NaN", "NaN"] or pd.isnull(correo) or pd.isna(correo):
            print("\033[31mEl contacto es nulo.\033[0m")
            results.append({"is_null": True})
            continue
        print(f"Buscando contacto: {df_pruebas['correo_cleaned'].iloc[n]}")
        result = (scrape_contact(driver, df_pruebas["correo_cleaned"].iloc[n]))
        results.append(result)
    display(results)



### Extraccion de excel

In [ ]:
import json
def extract_info_excel(driver, df, search_column: str) -> pd.DataFrame:
    """
    Extrae información de contactos desde un archivo Excel y los busca en Google Contacts.
    
    Args:
        driver (uc.Chrome): Instancia del driver de Selenium.
        excel_file (str): Ruta del archivo Excel.
        sheet_name (str): Nombre de la hoja del Excel.
        search_column (str): Nombre de la columna que contiene los correos electrónicos.
        time_sleep (int): Tiempo de espera entre búsquedas. Default es 2 segundos.
    """

    results = []
    for correo in df[f"{search_column}_cleaned"]:
        if correo in ["", " ", None, "nan", "N/A", "NaN", "NaN"] or pd.isnull(correo) or pd.isna(correo):
            print("\033[31mEl contacto es nulo.\033[0m")
            results.append({"is_null": True})
            continue
        print("\033[32mBuscando contacto:", correo, "\033[0m")
        results.append(scrape_contact(driver, correo))
        time.sleep(time_sleep)

    # Convertir lista con json a DataFrame
    df_results = pd.DataFrame(results)
    return df_results


if __name__ == '__pruebas__':
    df_pruebas = pd.read_excel(excel_file, sheet_name=sheet_name)
    display(df_pruebas.head(8))
    df_pruebas["correo_cleaned"] = get_clean_emails_from_df(df, search_column, display_results=False) # Para pruebas hay que limpiar los correos
    display(extract_info_excel(driver, df_pruebas, search_column))


# Ejecutar

In [ ]:
df = pd.read_excel(excel_file, sheet_name=sheet_name)
create_folders("output/parts", "output/filled")  # Crear carpetas de salida

df[f"{search_column}_cleaned"] = get_clean_emails_from_df(df, search_column, display_results=False)  # Limpiar los correos
split_excel(df, "output/parts", split_n)  # Dividir el DataFrame en partes y guardarlas en Excel

print("He encontrado", len(os.listdir("output/parts")), "archivos")

# Quitamos los archivos que no queremos procesar
files = sorted(os.listdir("output/parts"), key=lambda f: int(re.search(r'\d+', f).group()))[start:end]
print(f"Procesando los archivos{len(files)}:", files)
# Para el nombre del archivo pondremos el nombre del primer archivo y el último
output_filename = f"df_part_{start}-{end}"
print("El archivo de salida será:", output_filename, "\n")

# Inicializamos el driver
driver = get_stable_driver()
make_full_screen(driver)
zoom_out(driver, percentage=75)
driver.get("https://contacts.google.com/")
# Iniciar sesión en Google Contacts
if auto_login:
    iniciar_sesion(driver, correo, user, password)
else:
    print("Inicia sesión en Google Contacts y presiona Enter...")
    input("Inicia sesión en Google Contacts y presiona Enter...")

for file in files:
    file_path = f"output/parts/{file}"
    excel_part = pd.read_excel(file_path, sheet_name=sheet_name)
    result_df = extract_info_excel(driver, excel_part, search_column)  # Extraer información de los contactos
    # Concatenamos el DataFrame original con el resultado
    result_df = pd.concat([excel_part, result_df], axis=1)
    # Guardar resultados en un nuevo archivo Excel   
    result_df.to_excel(f"output/filled/{file[:-5]}_filled.xlsx", index=False, sheet_name=sheet_name)
    print(f"Archivo creado: output/filled/{file[:-5]}_filled.xlsx")


# Juntamos todos los archivos en uno solo
result_files = sorted(os.listdir("output/filled"), key=lambda f: int(re.search(r'\d+', f).group()))
result_files = [f"output/filled/{file}" for file in result_files]
# Juntamos todos los archivos en uno solo
df_final = pd.concat([pd.read_excel(file) for file in result_files], ignore_index=True)
# Ponemos la columna is_null al final
is_null_col = df_final.pop("is_null")
df_final.insert(len(df_final.columns), "is_null", is_null_col)
# Guardar el DataFrame final en un archivo Excel
df_final.to_excel(f"output/{output_filename}_filled.xlsx", index=False, sheet_name=sheet_name)
print("He creado el archivo final:", f"output/{output_filename}_filled.xlsx")
